## Battle of Neighborhoods Final Project 


## Data Exploration for Gueens, New York City



In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.19.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


<a id='item1'></a>

In [6]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.

In [8]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [9]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [10]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [11]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [12]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [13]:
neighborhoods.head()




,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [15]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()



,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


Let's get the geographical coordinates of Queens.

In [16]:
address = 'Queens, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Queens are 40.6524927, -73.7914214158161.


As we did with all of New York City, let's visualizat Queens the neighborhoods in it.

In [17]:
# create map of Manhattan using latitude and longitude values
map_Queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Queens)  
    
map_Queens

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'ABCA1MAERQOHGUMEQ3F5V5KCVIVFNL54LILF45DJS51AGT40' # your Foursquare ID
CLIENT_SECRET = 'DFZCDBC4KP2QYLLC30NXA01WUEF0VW4IX5P5VXV0JEWB44UK'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ABCA1MAERQOHGUMEQ3F5V5KCVIVFNL54LILF45DJS51AGT40
CLIENT_SECRET:DFZCDBC4KP2QYLLC30NXA01WUEF0VW4IX5P5VXV0JEWB44UK


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [19]:
queens_data.loc[0, 'Neighborhood']

'Astoria'

Get the neighborhood's latitude and longitude values.

In [20]:
neighborhood_latitude = queens_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = queens_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = queens_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Astoria are 40.76850859335492, -73.91565374304234.


#### Now, let's get the top 100 venues that are in Astoria within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=ABCA1MAERQOHGUMEQ3F5V5KCVIVFNL54LILF45DJS51AGT40&client_secret=DFZCDBC4KP2QYLLC30NXA01WUEF0VW4IX5P5VXV0JEWB44UK&v=20180605&ll=40.76850859335492,-73.91565374304234&radius=500&limit=100'

Send the GET request and examine the resutls

In [75]:
results = requests.get(url).json()
#results

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Favela Grill,Brazilian Restaurant,40.767348,-73.917897
1,Orange Blossom,Gourmet Shop,40.769856,-73.917012
2,Titan Foods Inc.,Gourmet Shop,40.769198,-73.919253
3,CrossFit Queens,Gym,40.769404,-73.918977
4,Simply Fit Astoria,Gym,40.769114,-73.912403


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Queens

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )



Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


#### Let's check the size of the resulting dataframe

In [28]:
print(queens_venues.shape)
queens_venues.head()

(2128, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym


Let's check how many venues were returned for each neighborhood

In [29]:
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,17,17,17,17,17,17
Astoria,100,100,100,100,100,100
Astoria Heights,10,10,10,10,10,10
Auburndale,19,19,19,19,19,19
Bay Terrace,41,41,41,41,41,41
Bayside,68,68,68,68,68,68
Bayswater,3,3,3,3,3,3
Beechhurst,17,17,17,17,17,17
Bellaire,12,12,12,12,12,12


#### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(queens_venues['Venue Category'].unique())))

There are 268 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [31]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[-1]] + list(queens_onehot.columns[:-1])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [32]:
queens_onehot.shape

(2128, 268)

In [76]:
venue_list=queens_onehot.columns.values.tolist() 
#venue_list

In [58]:
important_catageory_list=[
    'Neighborhood',
    'Art Museum', 'Arts & Entertainment',
 'Asian Restaurant',
 'Beach',
 'Cantonese Restaurant',
 'Chinese Restaurant',
 'Clothing Store',
 'Concert Hall',
 'Department Store',
 'Dim Sum Restaurant',
 'Diner',
 'Fast Food Restaurant',
 'General Entertainment',
 'Gift Shop',
 'Hotel',
 'Japanese Restaurant',
 'Korean Restaurant',
 "Men's Store",
 'Monument / Landmark',
 'Museum',
 'Park',
 'Restaurant',
 'Seafood Restaurant',
 'Shanghai Restaurant',
 'Shoe Store',
 'Shopping Mall',
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 'Theater',]

important_catageory_list
queens_common = queens_onehot[important_catageory_list]
queens_common.head()

,Neighborhood,Art Museum,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Concert Hall,Department Store,Dim Sum Restaurant,Diner,Fast Food Restaurant,General Entertainment,Gift Shop,Hotel,Japanese Restaurant,Korean Restaurant,Men's Store,Monument / Landmark,Museum,Park,Restaurant,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Sushi Restaurant,Taiwanese Restaurant,Theater
0,Astoria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Astoria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Astoria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Astoria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Astoria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
queens_grouped = queens_common.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Art Museum,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Concert Hall,Department Store,Dim Sum Restaurant,Diner,Fast Food Restaurant,General Entertainment,Gift Shop,Hotel,Japanese Restaurant,Korean Restaurant,Men's Store,Monument / Landmark,Museum,Park,Restaurant,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Sushi Restaurant,Taiwanese Restaurant,Theater
0,Arverne,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Astoria,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
2,Astoria Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Auburndale,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bay Terrace,0.000000,0.000000,0.024390,0.000000,0.000000,0.000000,0.121951,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.024390,0.000000,0.000000,0.000000,0.024390,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.024390,0.000000,0.000000,0.000000
5,Bayside,0.000000,0.000000,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.014706,0.014706,0.000000,0.029412,0.000000,0.000000
6,Bayswater,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Beechhurst,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000
8,Bellaire,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Belle Harbor,0.000000,0.000000,0.000000,0.166667,0.000000,0.055556,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [60]:
queens_grouped.shape

(81, 30)

#### Let's print each neighborhood along with the top 5 most common venues

In [62]:
num_top_venues = 5

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
                  venue  freq
0                 Beach  0.06
1            Art Museum  0.00
2   Japanese Restaurant  0.00
3  Taiwanese Restaurant  0.00
4      Sushi Restaurant  0.00


----Astoria----
                 venue  freq
0   Seafood Restaurant  0.04
1  Japanese Restaurant  0.01
2     Sushi Restaurant  0.01
3                Diner  0.01
4    Korean Restaurant  0.01


----Astoria Heights----
                  venue  freq
0            Art Museum   0.0
1   Japanese Restaurant   0.0
2  Taiwanese Restaurant   0.0
3      Sushi Restaurant   0.0
4         Shopping Mall   0.0


----Auburndale----
                  venue  freq
0  Fast Food Restaurant  0.05
1     Korean Restaurant  0.05
2            Art Museum  0.00
3   Japanese Restaurant  0.00
4  Taiwanese Restaurant  0.00


----Bay Terrace----
              venue  freq
0    Clothing Store  0.12
1  Asian Restaurant  0.02
2     Shopping Mall  0.02
3        Shoe Store  0.02
4         Gift Shop  0.02


----Bayside----
         

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Beach,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Concert Hall,Department Store
1,Astoria,Seafood Restaurant,Sushi Restaurant,Korean Restaurant,Japanese Restaurant,Diner,Theater,Fast Food Restaurant,Arts & Entertainment,Asian Restaurant,Beach
2,Astoria Heights,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Concert Hall,Department Store
3,Auburndale,Korean Restaurant,Fast Food Restaurant,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Chinese Restaurant,Clothing Store
4,Bay Terrace,Clothing Store,Shopping Mall,Shoe Store,Asian Restaurant,Men's Store,Gift Shop,Theater,Fast Food Restaurant,Arts & Entertainment,Beach
5,Bayside,Sushi Restaurant,Shoe Store,Shanghai Restaurant,Asian Restaurant,Korean Restaurant,Theater,Fast Food Restaurant,Arts & Entertainment,Beach,Cantonese Restaurant
6,Bayswater,Park,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Concert Hall
7,Beechhurst,Shopping Mall,Chinese Restaurant,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Clothing Store,Concert Hall
8,Bellaire,Chinese Restaurant,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Clothing Store,Concert Hall,Department Store
9,Belle Harbor,Beach,Chinese Restaurant,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Cantonese Restaurant,Clothing Store,Concert Hall,Department Store


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [66]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [67]:
queens_merged = queens_data

# add clustering labels
queens_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,1,Seafood Restaurant,Sushi Restaurant,Korean Restaurant,Japanese Restaurant,Diner,Theater,Fast Food Restaurant,Arts & Entertainment,Asian Restaurant,Beach
1,Queens,Woodside,40.746349,-73.901842,1,Restaurant,Asian Restaurant,Chinese Restaurant,Diner,Japanese Restaurant,Fast Food Restaurant,Theater,General Entertainment,Arts & Entertainment,Beach
2,Queens,Jackson Heights,40.751981,-73.882821,1,Shoe Store,Clothing Store,Diner,Sushi Restaurant,Asian Restaurant,Theater,General Entertainment,Arts & Entertainment,Beach,Cantonese Restaurant
3,Queens,Elmhurst,40.744049,-73.881656,1,Chinese Restaurant,Sushi Restaurant,Park,Theater,General Entertainment,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Clothing Store
4,Queens,Howard Beach,40.654225,-73.838138,1,Clothing Store,Fast Food Restaurant,Sushi Restaurant,Seafood Restaurant,Chinese Restaurant,Concert Hall,Japanese Restaurant,Diner,Theater,General Entertainment


Finally, let's visualize the resulting clusters

In [68]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [70]:
queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Flushing,Chinese Restaurant,Korean Restaurant,Asian Restaurant,Hotel,Sushi Restaurant,Seafood Restaurant,Cantonese Restaurant,Taiwanese Restaurant,Fast Food Restaurant,Arts & Entertainment
12,East Elmhurst,Chinese Restaurant,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Clothing Store,Concert Hall,Department Store
31,Jamaica Center,Clothing Store,Chinese Restaurant,Concert Hall,Department Store,Men's Store,Japanese Restaurant,Theater,General Entertainment,Arts & Entertainment,Asian Restaurant
65,Bellaire,Chinese Restaurant,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Clothing Store,Concert Hall,Department Store
66,North Corona,Hotel,Museum,Fast Food Restaurant,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Chinese Restaurant,Clothing Store
68,Jamaica Hills,Fast Food Restaurant,Shopping Mall,Asian Restaurant,Seafood Restaurant,Chinese Restaurant,Theater,General Entertainment,Arts & Entertainment,Beach,Cantonese Restaurant


#### Cluster 2

In [71]:
queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Seafood Restaurant,Sushi Restaurant,Korean Restaurant,Japanese Restaurant,Diner,Theater,Fast Food Restaurant,Arts & Entertainment,Asian Restaurant,Beach
1,Woodside,Restaurant,Asian Restaurant,Chinese Restaurant,Diner,Japanese Restaurant,Fast Food Restaurant,Theater,General Entertainment,Arts & Entertainment,Beach
2,Jackson Heights,Shoe Store,Clothing Store,Diner,Sushi Restaurant,Asian Restaurant,Theater,General Entertainment,Arts & Entertainment,Beach,Cantonese Restaurant
3,Elmhurst,Chinese Restaurant,Sushi Restaurant,Park,Theater,General Entertainment,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Clothing Store
4,Howard Beach,Clothing Store,Fast Food Restaurant,Sushi Restaurant,Seafood Restaurant,Chinese Restaurant,Concert Hall,Japanese Restaurant,Diner,Theater,General Entertainment
5,Corona,Chinese Restaurant,Restaurant,Park,Theater,General Entertainment,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Clothing Store
6,Forest Hills,Park,Asian Restaurant,Shoe Store,Chinese Restaurant,Theater,General Entertainment,Arts & Entertainment,Beach,Cantonese Restaurant,Clothing Store
7,Kew Gardens,Chinese Restaurant,Diner,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Clothing Store,Concert Hall
8,Richmond Hill,Clothing Store,Park,Diner,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Chinese Restaurant
10,Long Island City,Hotel,Restaurant,Arts & Entertainment,Chinese Restaurant,General Entertainment,Park,Art Museum,Seafood Restaurant,Shanghai Restaurant,Sushi Restaurant


#### Cluster 3

In [72]:
queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Utopia,Chinese Restaurant,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Clothing Store,Concert Hall,Department Store


#### Cluster 4

In [73]:
queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,Floral Park,Theater,Gift Shop,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Chinese Restaurant,Clothing Store,Concert Hall,Department Store


#### Cluster 5

In [74]:
queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Murray Hill,Korean Restaurant,Shoe Store,Japanese Restaurant,Theater,General Entertainment,Arts & Entertainment,Asian Restaurant,Beach,Cantonese Restaurant,Chinese Restaurant
